In [51]:
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist
from bs4 import BeautifulSoup
import pandas as pd
import time
from webdriver_manager.chrome import ChromeDriverManager

In [52]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324


[WDM] - Driver [C:\Users\ese12\.wdm\drivers\chromedriver\win32\88.0.4324.96\chromedriver.exe] found in cache


In [53]:
#scraping for news
def nasa():
        url = 'https://mars.nasa.gov/news/'
        browser.visit(url)
        time.sleep(2)

        html = browser.html
        soup = BeautifulSoup(html, 'html.parser')

        news_ttl=[]
        for div in soup.find_all('div', attrs={'class':'content_title'}):
            news_ttl.append(div.find('a'))
        news_title=news_ttl[1].text

        news=soup.find('div', attrs={'class':'article_teaser_body'}).text
        return(news_title,news)

In [54]:
news_title,news_p=nasa()
print(f"News Title :{news_title}")
print(f"News Paragraph :{news_p}")

News Title :NASA's Perseverance Pays Off Back Home
News Paragraph :Even as the Perseverance rover approaches Mars, technology on board is paying off on Earth.


In [55]:
  #scraping for photo
    def marsPhoto():
        url = 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.html'
        browser.visit(url)
        time.sleep(2)

        html = browser.html
        soup = BeautifulSoup(html, 'html.parser')

        image_url=soup.find('img', attrs={'class':'thumb'})
        featured_image_url=f"https://www.jpl.nasa.gov{image_url['src']}"

        return(featured_image_url)

In [56]:
featured_image_url='https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.html'
print(featured_image_url)

https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.html


In [57]:
#scraping for facts
def marsFact():
        url='https://space-facts.com/mars/'
        tables = pd.read_html(url)
        time.sleep(3)
        df=tables[0].reset_index(drop=True)
        df.columns = ['Attributes','Value']
        df['Attributes'] = [x.strip().replace(':', '') for x in df['Attributes']]
        df['Attributes'] = [x.strip().replace(' ', '') for x in df['Attributes']]
        df=df.set_index('Attributes').T.to_dict('records')[0]
        # df=df.to_dict('records')
        return df

In [58]:
mars_facts_url = 'https://space-facts.com/mars/'
facts_table = pd.read_html(mars_facts_url)
facts_table

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers,
   Mars - Earth Comparison             Mars            Earth
 0               Diameter:         6,779 km        12,742 km
 1                   Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 2                  Moons:                2                1
 3      Distance from Sun:   227,943,824 km   149,598,262 km
 4         Length of Year:   687 Earth days      365.24 days
 5            Temperature:     -87 to -5 °C      -88 to 58°C,
           

In [59]:
df = facts_table[0]
df.columns = ["Parameters", "Values"]
df.head()

,Parameters,Values
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"


In [60]:
#scraping for hemisphere
def marsHemisphere(name):
        url=f'https://astrogeology.usgs.gov/search/map/Mars/Viking/{name}'
        browser.visit(url)
        time.sleep(2)
        html = browser.html
        soup = BeautifulSoup(html, 'html.parser')
        img=soup.find("div", attrs={"class":"wide-image-wrapper"}).find('li').find('a').get('href')
        title=soup.find("h2", attrs={"class":"title"}).text
        return img,title

In [61]:
hemisphere_image_urls=[]
image_links=['cerberus_enhanced','schiaparelli_enhanced','syrtis_major_enhanced','valles_marineris_enhanced']
for image_link in range(0,len(image_links)):
    img_url,title=marsHemisphere(image_links[image_link])
    hemisphere={
       'title':title,
       'img_url':img_url
        }
    hemisphere_image_urls.append(hemisphere)

In [62]:
hemisphere_image_urls


[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]

In [63]:
browser.quit()